In [ ]:
# Install required libraries
!pip install gdown opencv-python scikit-image

import os
import cv2
import gdown
import zipfile
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

# === Step 1: Get dataset links from user ===
suim_link = input("Enter Google Drive link for SUIM dataset: ").strip()
uieb_link = input("Enter Google Drive link for UIEB dataset: ").strip()

# Function to download from Google Drive
def download_and_extract(gdrive_link, output_dir):
    # Extract file ID from Google Drive link
    if "id=" in gdrive_link:
        file_id = gdrive_link.split("id=")[1]
    elif "/d/" in gdrive_link:
        file_id = gdrive_link.split("/d/")[1].split("/")[0]
    else:
        raise ValueError("Invalid Google Drive link format.")

    gdown.download(f"https://drive.google.com/uc?id={file_id}", "temp.zip", quiet=False)

    with zipfile.ZipFile("temp.zip", 'r') as zip_ref:
        zip_ref.extractall(output_dir)
    os.remove("temp.zip")
    print(f"Extracted to: {output_dir}")

# Download datasets
download_and_extract(suim_link, "SUIM")
download_and_extract(uieb_link, "UIEB")

# === Step 2: Image enhancement methods ===
def histogram_equalization(img):
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

def clahe_enhancement(img):
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_yuv[:,:,0] = clahe.apply(img_yuv[:,:,0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

def ugan_enhancement(img):
    # Placeholder: Replace with your trained UGAN model inference
    # For now, we return the input image
    return img

# === Step 3: PSNR & SSIM computation ===
def evaluate_dataset(dataset_path):
    psnr_results = {"Input": [], "HE": [], "CLAHE": [], "UGAN": []}
    ssim_results = {"Input": [], "HE": [], "CLAHE": [], "UGAN": []}

    # Assuming dataset_path has 'input' and 'reference' subfolders
    input_dir = os.path.join(dataset_path, "input")
    ref_dir = os.path.join(dataset_path, "reference")
    img_files = sorted(os.listdir(input_dir))

    for fname in img_files:
        img_in = cv2.imread(os.path.join(input_dir, fname))
        img_ref = cv2.imread(os.path.join(ref_dir, fname))

        if img_in is None or img_ref is None:
            continue

        # Methods
        he_img = histogram_equalization(img_in)
        clahe_img = clahe_enhancement(img_in)
        ugan_img = ugan_enhancement(img_in)

        # Calculate metrics
        for method_name, method_img in zip(
            ["Input", "HE", "CLAHE", "UGAN"],
            [img_in, he_img, clahe_img, ugan_img]
        ):
            p = psnr(img_ref, method_img, data_range=255)
            s = ssim(cv2.cvtColor(img_ref, cv2.COLOR_BGR2GRAY),
                     cv2.cvtColor(method_img, cv2.COLOR_BGR2GRAY),
                     data_range=255)
            psnr_results[method_name].append(p)
            ssim_results[method_name].append(s)

    # Average results
    for method in psnr_results:
        psnr_results[method] = np.mean(psnr_results[method])
        ssim_results[method] = np.mean(ssim_results[method])

    return psnr_results, ssim_results

# === Step 4: Run evaluations ===
print("\nEvaluating SUIM dataset...")
suim_psnr, suim_ssim = evaluate_dataset("SUIM")
print("SUIM PSNR (dB):", suim_psnr)
print("SUIM SSIM:", suim_ssim)

print("\nEvaluating UIEB dataset...")
uieb_psnr, uieb_ssim = evaluate_dataset("UIEB")
print("UIEB PSNR (dB):", uieb_psnr)
print("UIEB SSIM:", uieb_ssim)
